In [2]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import matplotlib.pyplot as plt

# Define paths
MODEL_NAME = "gpt2-medium"
TOKENIZER = GPT2Tokenizer.from_pretrained(MODEL_NAME)
CONFIG = GPT2Config.from_pretrained(MODEL_NAME)

# Ensure PAD token exists
TOKENIZER.pad_token = TOKENIZER.eos_token

# Load sentence transformer model for similarity comparison
SIMILARITY_MODEL = SentenceTransformer("all-MiniLM-L6-v2")


class ProductDescriptionDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=128):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        product_name = str(self.data.iloc[idx].get("product_name", "Unknown"))
        category = str(self.data.iloc[idx].get("category", "Unknown"))
        price = str(self.data.iloc[idx].get("discounted_price", "Unknown"))
        link = str(self.data.iloc[idx].get("product_link", "Unknown"))
        about_product_review = str(self.data.iloc[idx].get("review_content", "No Review"))

        input_text = f"{product_name}. Category: {category}. Price: {price}. Link: {link}. About Product Review: {about_product_review}"

        encoded = self.tokenizer.encode_plus(
            input_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        return encoded["input_ids"].squeeze(0), encoded["attention_mask"].squeeze(0)


def recommend_products(user_description, dataset, similarity_model, top_n=5):
    """
    Recommend top N products based on the user's product description.
    """
    # Compute embedding for user input
    user_embedding = similarity_model.encode(user_description, convert_to_tensor=True)

    product_texts = []
    product_details = []

    # Extract product details for similarity matching
    for idx in range(len(dataset.data)):
        product_name = dataset.data.iloc[idx]["product_name"]
        category = dataset.data.iloc[idx]["category"]
        price = dataset.data.iloc[idx]["discounted_price"]
        link = dataset.data.iloc[idx]["product_link"]
        about_review = dataset.data.iloc[idx]["review_content"]

        product_text = f"{product_name}. Category: {category}. Price: {price}. About Review: {about_review}"
        product_texts.append(product_text)
        product_details.append((product_name, category, price, link))  # Storing details for results

    # Compute embeddings for all product descriptions
    product_embeddings = similarity_model.encode(product_texts, convert_to_tensor=True)

    # Compute cosine similarity
    similarities = util.pytorch_cos_sim(user_embedding, product_embeddings)[0]

    # Get top N recommendations
    top_indices = torch.argsort(similarities, descending=True)[:top_n]

    recommendations = []
    for idx in top_indices:
        product_name, category, price, link = product_details[idx]
        recommendations.append(
            {
                "Product Name": product_name,
                "Category": category,
                "Price": price,
                "Product Link": link,
                "Similarity Score": similarities[idx].item(),
            }
        )

    return recommendations

# Load dataset
csv_path = "/content/amazon.csv"
train_dataset = ProductDescriptionDataset(csv_path, TOKENIZER)

# Load the YOLOv5 model (you can replace 'yolov5s' with your custom model)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


def display_results(user_description):
    # Get recommended products
    recommended_products = recommend_products(user_description, train_dataset, SIMILARITY_MODEL)

    # Display recommendations
    print("\nTop Recommended Products:")
    for idx, product in enumerate(recommended_products, 1):
        print(f"\n{idx}. {product['Product Name']}")  # Fixed key name
        print(f"   - Category: {product['Category']}")
        print(f"   - Price: {product['Price']}")
        print(f"   - Link: {product['Product Link']}")
        print(f"   - Similarity Score: {product['Similarity Score']:.4f}")



Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-2-22 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
import torch
import cv2
import numpy as np
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from ultralytics import YOLO

# Load YOLOv8 model for object detection
detector = YOLO("yolov8n.pt")  # Using the smallest version for speed

# Load BLIP model for image captioning
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def detect_products(image):
    """Detect products in the image using YOLOv8."""
    results = detector(image)
    detected_objects = []

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            detected_objects.append((x1, y1, x2, y2))

    return detected_objects

def generate_caption(image):
    """Generate a product-focused description using BLIP."""
    inputs = processor(images=image, return_tensors="pt")
    output = caption_model.generate(**inputs)
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

def process_image(image_path, output_path="output.jpg"):
    """Process an image: detect products, draw bounding boxes, and generate captions."""
    image = Image.open(image_path).convert("RGB")
    image_np = np.array(image)

    # Detect products
    boxes = detect_products(image_np)

    # Draw bounding boxes
    for x1, y1, x2, y2 in boxes:
        cv2.rectangle(image_np, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Save processed image
    output_image = Image.fromarray(image_np)
    output_image.save(output_path)

    # Generate and return product captions
    captions = []
    for i, (x1, y1, x2, y2) in enumerate(boxes):
        cropped_img = image.crop((x1, y1, x2, y2))
        caption = generate_caption(cropped_img)
        captions.append(f"Product {i+1}: {caption}")

    return captions

In [11]:
# Get user input for product description
user_input_type = input("Enter 1 for product description and 2 for image upload: ")
if user_input_type == "1":
    user_description = input("Enter Product Description: ")
    display_results(user_description)
else:
    # Load an image (replace with your uploaded image)
    img_path = input("Enter image path: ")
    captions = process_image(img_path)
    for caption in captions:
        display_results(caption)

Enter 1 for product description and 2 for image upload: 2
Enter image path: /content/images.jpeg

0: 320x640 1 bicycle, 72.4ms
Speed: 2.6ms preprocess, 72.4ms inference, 6.3ms postprocess per image at shape (1, 3, 320, 640)

Top Recommended Products:

1. DIGITEK® (DTR-200MT) (18 CM) Portable & Flexible Mini Tripod with Mobile Holder & 360 Degree Ball Head, For Smart Phones, Compact Cameras, GoPro, Maximum Operating Height: 7.87 Inch, Maximum Load Upto: 1 kgs
   - Category: Electronics|Cameras&Photography|Accessories|Tripods&Monopods|TripodLegs
   - Price: ₹349
   - Link: https://www.amazon.in/DIGITEK-Portable-Flexible-Compact-Operating/dp/B08B6XWQ1C/ref=sr_1_269?qid=1672903008&s=computers&sr=1-269
   - Similarity Score: 0.5415

2. SYVO WT 3130 Aluminum Tripod (133CM), Universal Lightweight Tripod with Mobile Phone Holder Mount & Carry Bag for All Smart Phones, Gopro, Cameras - Brown
   - Category: Electronics|Cameras&Photography|Accessories|Tripods&Monopods|Tabletop&TravelTripods
   - 